In [15]:
import csv
header=['Input', 'Prediction']


with open('n2_stf_test.csv', 'w', encoding='UTF8') as f:
    writer = csv.writer(f)
    # write the header
    writer.writerow(header)


    groups = [
        ['R', 'K', 'H'], #pos charge
        ['D', 'E'], #neg charge
        ['S', 'T', 'N', 'Q'], 
        ['F', 'W', 'Y'], #aromatic
        ['C'], #special
        ['G'], #special
        ['P'], #special
        ['A', 'I', 'L', 'M', 'V'] #hydrophobic
    ]

    start = 1
    an = 'NP_077719.2'

    # NP_000426.2_neurogenic_locus_notch_homolog_protein_3_precursor_[Homo_sapiens] = n3
    # NP_060087.3 = n1
    # NP_077719.2 = n2
    # NP_004548.3 = n4


    filename = 'trees_ML/algn/notch2/re_new/notch2.fa'
    #filename = 'n3_no_gaps_no_rept.fa'

    with open(filename, 'r') as inp:
        lines = inp.readlines()

    msa = dict()

    for l in lines:
        if l.startswith('>'):
            k = l.split()[0][1:]
            msa[k] = ''
        else:
            msa[k] += l.strip('\n')


    # get rid of gaps - take index of AAs that are not gaps in human
    an_pos = list()

    for i in range(0, len(msa[an])):
        if msa[an][i] != '-':
            an_pos.append(i)


    # dict with no human gaps
    an_msa = dict()
    for k in msa:
        an_msa[k] = ''
        for i in an_pos:
            an_msa[k] += msa[k][i]


    filename_mut = 'variants/notch2_table2'


    with open(filename_mut, 'r') as inp:
        lines = [l.strip('\n') for l in inp.readlines()]

    for l in lines:
        statement = {
            'bullseye': {'s': 0, 'w': 0},
            'other_group': {'s': 0, 'w': 0},
            'same_group_as_ref': {'s': 0, 'w': 0},
            'same_group_as_alt': {'s': 0, 'w': 0}
        }
        pos = l[1:-1]
        pos = int(pos)-start
        whole_pos = an_pos[pos]
        alt = l[-1]
        ref = an_msa[an][pos]
        print(l)
        if ref != l[0]:
            raise Exception(l)


        # define AA group for ref and alt 
        for i in range(0, len(groups)):
            if ref in groups[i]:
                group_i = i
        for i in range(0, len(groups)):
            if alt in groups[i]:
                group_a = i

        what_do_we_have_here = dict()
        what_do_we_have_here_an = dict()

        for k in an_msa:
            #take pos-1,pos,pos+1
            #what_do_we_have_here_an[k] = an_msa[k][pos-1:pos+2]
            #what_do_we_have_here[k] = msa[k][whole_pos-1:whole_pos+2]

            if pos == 0:
                what_do_we_have_here_an[k] = an_msa[k][pos:pos+2]
                what_do_we_have_here[k] = msa[k][whole_pos:whole_pos+2]
                for k in what_do_we_have_here:
                    var = what_do_we_have_here[k][0] # pos from triplet (012)
                    if (var != ref) and (var != '-'):
                        strong = False
                        aa2 = what_do_we_have_here[k][1]
                        aan2 = what_do_we_have_here_an[k][1]
                        weak = True
                        #chack for same context when ref != current pos:
                        if (aa2 == what_do_we_have_here[an][1]):
                            weak = False
                        if not weak:
                            weak = True
                            if (aan2 == what_do_we_have_here_an[an][1]):
                                weak = False
                        if not weak:
                            strong = True
            #            for k1 in what_do_we_have_here:
            #                if k1 != k:
            #                    if (aa0 == what_do_we_have_here[k1][0]) and \
            #                    (aa2 == what_do_we_have_here[k1][2]):
            #                        weak = False
            #            if not weak:
            #                weak = True
            #                for k1 in what_do_we_have_here_an:
            #                    if k1 != k:
            #                        if (aan0 == what_do_we_have_here_an[k1][0]) and \
            #                        (aan2 == what_do_we_have_here_an[k1][2]):
            #                            weak = False
                        
                        # check if var is clinvar variant 
                        if var == alt:
                            if strong:
                                statement['bullseye']['s'] += 1
                            else:
                                statement['bullseye']['w'] += 1
                        # check for AA groups    
                        # same group as ref:    
                        if var in groups[group_i]:
                            if strong:
                                statement['same_group_as_ref']['s'] += 1
                            else:
                                statement['same_group_as_ref']['w'] += 1
                        # same as alt:        
                        elif var in groups[group_a]:
                            if strong:
                                statement['same_group_as_alt']['s'] += 1
                            else:
                                statement['same_group_as_alt']['w'] += 1
                        else:
                            if strong:
                                statement['other_group']['s'] += 1
                            else:
                                statement['other_group']['w'] += 1
            else:
                what_do_we_have_here_an[k] = an_msa[k][pos-1:pos+2]
                what_do_we_have_here[k] = msa[k][whole_pos-1:whole_pos+2]
            
                for k in what_do_we_have_here:
                    var = what_do_we_have_here[k][1] # pos from triplet (012)
                    if (var != ref) and (var != '-'):
                        strong = False
                        aa0 = what_do_we_have_here[k][0]
                        aa2 = what_do_we_have_here[k][2]
                        aan0 = what_do_we_have_here_an[k][0]
                        aan2 = what_do_we_have_here_an[k][2]
                        weak = True
                        #chack for same context when ref != current pos:
                        if (aa0 == what_do_we_have_here[an][0]) and \
                        (aa2 == what_do_we_have_here[an][2]):
                            weak = False
                        if not weak:
                            weak = True
                            if (aan0 == what_do_we_have_here_an[an][0]) and \
                            (aan2 == what_do_we_have_here_an[an][2]):
                                weak = False
                        if not weak:
                            strong = True
            #            for k1 in what_do_we_have_here:
            #                if k1 != k:
            #                    if (aa0 == what_do_we_have_here[k1][0]) and \
            #                    (aa2 == what_do_we_have_here[k1][2]):
            #                        weak = False
            #            if not weak:
            #                weak = True
            #                for k1 in what_do_we_have_here_an:
            #                    if k1 != k:
            #                        if (aan0 == what_do_we_have_here_an[k1][0]) and \
            #                        (aan2 == what_do_we_have_here_an[k1][2]):
            #                            weak = False
                        
                        # check if var is clinvar variant 
                        if var == alt:
                            if strong:
                                statement['bullseye']['s'] += 1
                            else:
                                statement['bullseye']['w'] += 1
                        # check for AA groups    
                        # same group as ref:    
                        if var in groups[group_i]:
                            if strong:
                                statement['same_group_as_ref']['s'] += 1
                            else:
                                statement['same_group_as_ref']['w'] += 1
                        # same as alt:        
                        elif var in groups[group_a]:
                            if strong:
                                statement['same_group_as_alt']['s'] += 1
                            else:
                                statement['same_group_as_alt']['w'] += 1
                        else:
                            if strong:
                                statement['other_group']['s'] += 1
                            else:
                                statement['other_group']['w'] += 1
        benignity = False
        for t in statement:
            if statement[t]['s'] > 0:
                print('Strong: ' + t + ' (' + str(statement[t]['s']) + ')')
                benignity = True
            elif statement[t]['w'] > 0:
                print('Weak: ' + t + ' (' + str(statement[t]['w']) + ')')
                benignity = True
        if not benignity:
            data = (l,'Pathogenic')
            writer.writerow(data)
            print('Pathogenic')
        elif statement['bullseye']['s'] > 0:
            data = (l,'Benign')
            writer.writerow(data)
            print('Benign')
        elif statement['same_group_as_alt']['s'] > 0:
            data = (l,'Benign?')
            writer.writerow(data)
            print('Benign (?)')
        elif statement['other_group']['s'] > 0:
            data = (l,'Benign??')
            writer.writerow(data)
            print('Benign (??)')
        elif statement['same_group_as_ref']['s'] > 0:
            data = (l,'Pathogenic?')
            writer.writerow(data)
            print('Pathogenic (?)')
        else:
            data = (l,'Unknown')
            writer.writerow(data)
            print('No simple answer')
        print('\n###\n')

L2408H
Strong: other_group (168)
Weak: same_group_as_ref (152)
Weak: same_group_as_alt (384)
Benign (??)

###

P2359A
Strong: bullseye (431)
Strong: other_group (2714)
Strong: same_group_as_alt (431)
Benign

###

Y2160C
Weak: same_group_as_ref (10196)
No simple answer

###

P2157T
Pathogenic

###

V2075M
Strong: bullseye (2687)
Weak: other_group (109)
Strong: same_group_as_ref (2988)
Benign

###

M2042T
Strong: same_group_as_ref (5922)
Pathogenic (?)

###

H2032N
Strong: other_group (610)
Benign (??)

###

H1882Y
Pathogenic

###

S1741L
Strong: other_group (619)
Strong: same_group_as_ref (324)
Strong: same_group_as_alt (423)
Benign (?)

###

I1689F
Strong: other_group (588)
Strong: same_group_as_ref (9491)
Benign (??)

###

V1667F
Strong: bullseye (5441)
Strong: other_group (17536)
Strong: same_group_as_ref (12644)
Strong: same_group_as_alt (5535)
Benign

###

V1667I
Strong: bullseye (8682)
Strong: other_group (23071)
Strong: same_group_as_ref (12644)
Benign

###

L1413H
Strong: bullse

In [28]:
statement

{'bullseye': {'s': 1100, 'w': 206},
 'other_group': {'s': 430, 'w': 0},
 'same_group_as_ref': {'s': 1841, 'w': 1632},
 'same_group_as_alt': {'s': 0, 'w': 0}}